<a href="https://colab.research.google.com/github/botkin92/ML_in_business/blob/lesson_4/HW_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel
from sklift.models import TwoModels
from sklift.models import ClassTransformation

from lightgbm import LGBMClassifier

In [ ]:
# !pip install scikit-uplift

1. скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention

In [ ]:
# df = pd.read_csv('data.csv')
# df.head()

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
df = pd.read_csv('MyDrive/Colab Notebooks/ML_in_business/data.csv')
df.head()

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0
3,9,675.83,1,0,Rural,1,Web,Discount,0
4,2,45.34,1,0,Urban,0,Web,Buy One Get One,0


In [ ]:
df.describe()

,recency,history,used_discount,used_bogo,is_referral,conversion
count,64000.000000,64000.000000,64000.000000,64000.000000,64000.000000,64000.000000
mean,5.763734,242.085656,0.551031,0.549719,0.502250,0.146781
std,3.507592,256.158608,0.497393,0.497526,0.499999,0.353890
min,1.000000,29.990000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,64.660000,0.000000,0.000000,0.000000,0.000000
50%,6.000000,158.110000,1.000000,1.000000,1.000000,0.000000
75%,9.000000,325.657500,1.000000,1.000000,1.000000,0.000000
max,12.000000,3345.930000,1.000000,1.000000,1.000000,1.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64000 entries, 0 to 63999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   recency        64000 non-null  int64  
 1   history        64000 non-null  float64
 2   used_discount  64000 non-null  int64  
 3   used_bogo      64000 non-null  int64  
 4   zip_code       64000 non-null  object 
 5   is_referral    64000 non-null  int64  
 6   channel        64000 non-null  object 
 7   offer          64000 non-null  object 
 8   conversion     64000 non-null  int64  
dtypes: float64(1), int64(5), object(3)
memory usage: 4.4+ MB


2. там поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.

In [ ]:
df.conversion.value_counts()

0    54606
1     9394
Name: conversion, dtype: int64

In [ ]:
df.offer.value_counts()

Buy One Get One    21387
Discount           21307
No Offer           21306
Name: offer, dtype: int64

In [ ]:
df.rename(columns={'conversion': 'target', 'offer': 'treatment'}, inplace=True)

In [ ]:
df['treatment'].replace({'Buy One Get One': 1, 'Discount': 1, 'No Offer': 0}, inplace=True)

In [ ]:
df.treatment.value_counts()

1    42694
0    21306
Name: treatment, dtype: int64

4. сделать feature engineering на ваше усмотрение (допускается свобода выбора методов)

In [ ]:
# zip_code, channel - onehot
# history - stdscale

In [ ]:
df = pd.get_dummies(df, columns=['zip_code', 'channel'], prefix=['zip_code', 'channel'])

In [ ]:
df[['history']] = StandardScaler().fit_transform(df[['history']])

In [ ]:
df.head()

,recency,history,used_discount,used_bogo,is_referral,treatment,target,zip_code_Rural,zip_code_Surburban,zip_code_Urban,channel_Multichannel,channel_Phone,channel_Web
0,10,-0.389003,1,0,0,1,0,0,1,0,0,1,0
1,6,0.339614,1,1,1,0,0,1,0,0,0,0,1
2,7,-0.239836,0,1,1,1,0,0,1,0,0,0,1
3,9,1.693278,1,0,1,1,0,1,0,0,0,0,1
4,2,-0.768068,1,0,0,1,0,0,0,1,0,0,1


3. сделать разбиение набора данных не тренировочную и тестовую выборки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', 1), df['target'], test_size=0.3, random_state=33)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [ ]:
treat_train = X_train['treatment']
treat_test = X_test['treatment']

X_train.drop('treatment', 1, inplace=True)
X_test.drop('treatment', 1, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """


In [ ]:
X_train.shape, X_test.shape

((44800, 11), (19200, 11))

5. провести uplift-моделирование 3 способами: одна модель с признаком коммуникации (S learner), модель с трансформацией таргета (трансформация классов п. 2. 1) и вариант с двумя независимыми моделями

In [ ]:
models_results = {
    'approach': [],
    'uplift@10%': [],
    'uplift@20%': [],
    'uplift@30%': []}

5.1 одна модель с признаком коммуникации

In [ ]:
sm = SoloModel(LGBMClassifier(n_estimators=10))
sm = sm.fit(X_train, y_train, treat_train)

uplift_sm = sm.predict(X_test)

In [ ]:
models_results['approach'].append('SoloModel')

for pr in [0.1, 0.2, 0.3]:
  sm_score = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=treat_test, strategy='by_group', k=pr)
  models_results[f'uplift@{pr*100:.0f}%'].append(sm_score)
  # print(f'uplift@{pr*100:.0f}%: {sm_score:.4f}')

5.2 модель с трансформацией таргета

In [ ]:
ct = ClassTransformation(LGBMClassifier(n_estimators=10))
ct = ct.fit(X_train, y_train, treat_train)

uplift_ct = ct.predict(X_test)

In [ ]:
models_results['approach'].append('ClassTransform')

for pr in [0.1, 0.2, 0.3]:
  sm_score = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=treat_test, strategy='by_group', k=pr)
  models_results[f'uplift@{pr*100:.0f}%'].append(sm_score)
  # print(f'uplift@{pr*100:.0f}%: {sm_score:.4f}')

5.3 две независимые модели

In [ ]:
tm = TwoModels(
    estimator_trmnt=LGBMClassifier(n_estimators=10), 
    estimator_ctrl=LGBMClassifier(n_estimators=10) 
)
tm = tm.fit(X_train, y_train, treat_train)

uplift_tm = tm.predict(X_test)

In [ ]:
models_results['approach'].append('TwoModel')

for pr in [0.1, 0.2, 0.3]:
  sm_score = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=treat_test, strategy='by_group', k=pr)
  models_results[f'uplift@{pr*100:.0f}%'].append(sm_score)

6. в конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% этих 3 моделей

In [ ]:
pd.DataFrame(data=models_results).sort_values('uplift@10%', ascending=False)

,approach,uplift@10%,uplift@20%,uplift@30%
2,TwoModel,0.095456,0.070059,0.062654
0,SoloModel,0.095358,0.073530,0.066539
1,ClassTransform,0.054655,0.062077,0.062501


7. построить модель UpliftTreeClassifier и попытаться описать словами полученное дерево